In [ ]:
{
 "cells": [
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# A simple example (unconstrained)\n",
    "\n",
    "## Define your problem\n",
    "\n",
    "Let's start with a simple problem of minimizing $x_1^4 + x_2^4$ with respect to $x_1$ and $x_2$.\n",
    "\n",
    "The mathematical problem statement is: \n",
    "\n",
    "\n",
    "$$\n",
    "\\underset{x_1, x_2 \\in \\mathbb{R}}{\\text{minimize}} \\quad x_1^4 + x_2^4\n",
    "$$\n",
    "\n",
    "We know the solution of this problem is $x_1=0$, and $x_2=0$.\n",
    "However, we start from an intial guess of $x_1=0.3$, and $x_2=0.3$ for the purposes of this tutorial.\n",
    "\n",
    "The problem is written in modOpt using the **Problem()** class as follows:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "from modopt.api import Problem\n",
    "\n",
    "\n",
    "class X4(Problem):\n",
    "    def initialize(self, ):\n",
    "        # Name your problem\n",
    "        self.problem_name = 'x^4'\n",
    "\n",
    "    def setup(self):\n",
    "        # Add design variables of your problem\n",
    "        self.add_design_variables('x',\n",
    "                                  shape=(2, ),\n",
    "                                  vals=np.array([.3, .3]))\n",
    "        self.add_objective('f')\n",
    "\n",
    "    def setup_derivatives(self):\n",
    "        # Declare objective gradient and its shape\n",
    "        self.declare_objective_gradient(wrt='x', )\n",
    "\n",
    "    # Compute the value of the objective with given design variable values\n",
    "    def compute_objective(self, dvs, obj):\n",
    "        obj['f'] = np.sum(dvs['x']**4)\n",
    "\n",
    "    def compute_objective_gradient(self, dvs, grad):\n",
    "        grad['x'] = 4 * dvs['x']**3"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Develop/Build your optimization algorithm\n",
    "\n",
    "Here we look at the **steepest descent** algorithm for unconstrained problems. \n",
    "We will later (in the next section) use it to solve the unconstrained  optimization problem defined above.\n",
    "\n",
    "For a general unconstrained optimization problem stated as: \n",
    "\n",
    "$$\n",
    "\\underset{x \\in \\mathbb{R^n}}{\\text{minimize}} \\quad f(x)\n",
    "$$\n",
    "\n",
    "the steepest descent algorithms computes the new iterate recursively by using the formula\n",
    "\n",
    "$$\n",
    "x_{k+1} = x_{k} - \\nabla f(x_k) .\n",
    "$$\n",
    "\n",
    "Given an initial guess $x_0$, we can write an optimizer using the steepest descent algorithm using the **Optimizer()** class in modOpt as follows:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import time\n",
    "from modopt.api import Optimizer\n",
    "\n",
    "\n",
    "class SteepestDescent(Optimizer):\n",
    "    def initialize(self):\n",
    "\n",
    "        # Name your algorithm\n",
    "        self.solver_name = 'steepest_descent'\n",
    "\n",
    "        self.obj = self.problem._compute_objective\n",
    "        self.grad = self.problem._compute_objective_gradient\n",
    "\n",
    "        self.options.declare('max_itr', default=1000, types=int)\n",
    "        self.options.declare('opt_tol', default=1e-5, types=float)\n",
    "\n",
    "        # Specify format of outputs available from your optimizer after each iteration\n",
    "        self.default_outputs_format = {\n",
    "            'itr': int,\n",
    "            'obj': float,\n",
    "            # for arrays from each iteration, shapes need to be declared\n",
    "            'x': (float, (self.problem.nx, )),\n",
    "            'opt': float,\n",
    "            'time': float,\n",
    "        }\n",
    "\n",
    "        # Enable user to specify, as a list, which among the available outputs\n",
    "        # need to be stored in memory and written to output files\n",
    "        self.options.declare('outputs',\n",
    "                             types=list,\n",
    "                             default=['itr', 'obj', 'x', 'opt', 'time'])\n",
    "\n",
    "    def solve(self):\n",
    "        nx = self.problem.nx\n",
    "        x = self.problem.x.get_data()\n",
    "        opt_tol = self.options['opt_tol']\n",
    "        max_itr = self.options['max_itr']\n",
    "\n",
    "        obj = self.obj\n",
    "        grad = self.grad\n",
    "\n",
    "        start_time = time.time()\n",
    "\n",
    "        # Setting intial values for initial iterates\n",
    "        x_k = x * 1.\n",
    "        f_k = obj(x_k)\n",
    "        g_k = grad(x_k)\n",
    "\n",
    "        # Iteration counter\n",
    "        itr = 0\n",
    "\n",
    "        # Optimality\n",
    "        opt = np.linalg.norm(g_k)\n",
    "\n",
    "        # Initializing outputs\n",
    "        self.update_outputs(itr=0,\n",
    "                            x=x_k,\n",
    "                            obj=f_k,\n",
    "                            opt=opt,\n",
    "                            time=time.time() - start_time)\n",
    "\n",
    "        while (opt > opt_tol and itr < max_itr):\n",
    "            itr_start = time.time()\n",
    "            itr += 1\n",
    "\n",
    "            # ALGORITHM STARTS HERE\n",
    "            # >>>>>>>>>>>>>>>>>>>>>\n",
    "\n",
    "            p_k = -g_k\n",
    "\n",
    "            x_k += p_k\n",
    "            f_k = obj(x_k)\n",
    "            g_k = grad(x_k)\n",
    "\n",
    "            opt = np.linalg.norm(g_k)\n",
    "\n",
    "            # <<<<<<<<<<<<<<<<<<<\n",
    "            # ALGORITHM ENDS HERE\n",
    "\n",
    "            # Append arrays inside outputs dict with new values from the current iteration\n",
    "            self.update_outputs(itr=itr,\n",
    "                                x=x_k,\n",
    "                                obj=f_k,\n",
    "                                opt=opt,\n",
    "                                time=time.time() - start_time)\n",
    "\n",
    "        # Run post-processing for the Optimizer() base class\n",
    "        self.run_post_processing()\n",
    "\n",
    "        end_time = time.time()\n",
    "        self.total_time = end_time - start_time"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The **Optimizer()** class records all the data needed using the `outputs` dictionary.\n",
    "\n",
    "## Solve your problem using your optimizer\n",
    "\n",
    "Now that we have modeled the problem and developed the optimizer, the task remaining is to solve the problem with the optimizer.\n",
    "For this, we need to set up our optimizer with the problem and pass in optimizer-specific parameters. \n",
    "Default values will be assumed if the optimizer parameters are not passed in."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Setting objective name as \"f\".\n",
      "\n",
      "----------------------------------------------------------------------------\n",
      "Derivative type | Calc norm  | FD norm    | Abs error norm | Rel error norm \n",
      "----------------------------------------------------------------------------\n",
      "\n",
      "Gradient        | 1.5274e-01 | 1.5274e-01 | 7.6367e-07     | 7.0710e-06    \n",
      "----------------------------------------------------------------------------\n",
      "\n",
      "\n",
      " \t ===============================\n",
      "\t ModOpt final iteration summary:\n",
      "\t ===============================\n",
      "\t Problem       : x^4\n",
      "\t Solver        : steepest_descent\n",
      "\t itr           : 100\n",
      "\t obj           : 2.8304561425587446e-06\n",
      "\t opt           : 0.0002321105148388349\n",
      "\t time          : 1.0574009418487549\n",
      "\t =====================================\n",
      "\n",
      "\n",
      "===============================\n",
      "     modOpt summary table:     \n",
      "===============================\n",
      " itr      obj      opt     time\n",
      "   0 1.62E-02 1.53E-01 7.99E-05\n",
      "   1 2.72E-03 4.00E-02 2.46E-02\n",
      "   2 1.44E-03 2.48E-02 3.17E-02\n",
      "   3 9.12E-04 1.77E-02 3.89E-02\n",
      "   4 6.38E-04 1.35E-02 4.64E-02\n",
      "   5 4.74E-04 1.08E-02 5.39E-02\n",
      "   6 3.68E-04 8.94E-03 6.02E-02\n",
      "   7 2.94E-04 7.56E-03 6.75E-02\n",
      "   8 2.41E-04 6.51E-03 7.30E-02\n",
      "   9 2.02E-04 5.69E-03 8.02E-02\n",
      "  10 1.71E-04 5.03E-03 8.60E-02\n",
      "  11 1.47E-04 4.49E-03 9.16E-02\n",
      "  12 1.28E-04 4.05E-03 9.81E-02\n",
      "  13 1.12E-04 3.67E-03 1.06E-01\n",
      "  14 9.95E-05 3.35E-03 1.16E-01\n",
      "  15 8.87E-05 3.07E-03 1.25E-01\n",
      "  16 7.96E-05 2.84E-03 1.33E-01\n",
      "  17 7.19E-05 2.63E-03 1.41E-01\n",
      "  18 6.52E-05 2.44E-03 1.49E-01\n",
      "  19 5.95E-05 2.28E-03 1.56E-01\n",
      "  20 5.45E-05 2.13E-03 1.62E-01\n",
      "  21 5.01E-05 2.00E-03 2.20E-01\n",
      "  22 4.62E-05 1.88E-03 2.37E-01\n",
      "  23 4.27E-05 1.78E-03 2.50E-01\n",
      "  24 3.96E-05 1.68E-03 2.60E-01\n",
      "  25 3.69E-05 1.59E-03 2.70E-01\n",
      "  26 3.44E-05 1.51E-03 2.86E-01\n",
      "  27 3.22E-05 1.44E-03 2.97E-01\n",
      "  28 3.02E-05 1.37E-03 3.08E-01\n",
      "  29 2.83E-05 1.31E-03 3.19E-01\n",
      "  30 2.67E-05 1.25E-03 3.29E-01\n",
      "  31 2.52E-05 1.19E-03 3.40E-01\n",
      "  32 2.38E-05 1.14E-03 3.47E-01\n",
      "  33 2.25E-05 1.10E-03 3.60E-01\n",
      "  34 2.13E-05 1.05E-03 3.71E-01\n",
      "  35 2.02E-05 1.01E-03 3.79E-01\n",
      "  36 1.92E-05 9.75E-04 3.89E-01\n",
      "  37 1.83E-05 9.40E-04 3.97E-01\n",
      "  38 1.74E-05 9.06E-04 4.04E-01\n",
      "  39 1.66E-05 8.74E-04 4.10E-01\n",
      "  40 1.58E-05 8.44E-04 4.19E-01\n",
      "  41 1.51E-05 8.16E-04 4.25E-01\n",
      "  42 1.45E-05 7.89E-04 4.33E-01\n",
      "  43 1.39E-05 7.64E-04 4.42E-01\n",
      "  44 1.33E-05 7.40E-04 4.49E-01\n",
      "  45 1.28E-05 7.18E-04 4.57E-01\n",
      "  46 1.22E-05 6.96E-04 4.65E-01\n",
      "  47 1.18E-05 6.76E-04 4.73E-01\n",
      "  48 1.13E-05 6.56E-04 4.80E-01\n",
      "  49 1.09E-05 6.38E-04 4.87E-01\n",
      "  50 1.05E-05 6.20E-04 4.94E-01\n",
      "  51 1.01E-05 6.03E-04 5.03E-01\n",
      "  52 9.75E-06 5.87E-04 5.10E-01\n",
      "  53 9.41E-06 5.72E-04 5.17E-01\n",
      "  54 9.09E-06 5.57E-04 5.24E-01\n",
      "  55 8.78E-06 5.43E-04 5.32E-01\n",
      "  56 8.49E-06 5.29E-04 5.40E-01\n",
      "  57 8.22E-06 5.16E-04 5.47E-01\n",
      "  58 7.95E-06 5.04E-04 5.55E-01\n",
      "  59 7.70E-06 4.92E-04 5.62E-01\n",
      "  60 7.46E-06 4.80E-04 5.70E-01\n",
      "  61 7.24E-06 4.69E-04 5.77E-01\n",
      "  62 7.02E-06 4.59E-04 5.84E-01\n",
      "  63 6.81E-06 4.48E-04 5.92E-01\n",
      "  64 6.61E-06 4.39E-04 6.00E-01\n",
      "  65 6.42E-06 4.29E-04 6.08E-01\n",
      "  66 6.24E-06 4.20E-04 6.18E-01\n",
      "  67 6.06E-06 4.11E-04 6.30E-01\n",
      "  68 5.90E-06 4.02E-04 6.40E-01\n",
      "  69 5.74E-06 3.94E-04 6.52E-01\n",
      "  70 5.58E-06 3.86E-04 6.61E-01\n",
      "  71 5.43E-06 3.79E-04 6.76E-01\n",
      "  72 5.29E-06 3.71E-04 6.86E-01\n",
      "  73 5.16E-06 3.64E-04 6.98E-01\n",
      "  74 5.03E-06 3.57E-04 7.13E-01\n",
      "  75 4.90E-06 3.50E-04 7.26E-01\n",
      "  76 4.78E-06 3.44E-04 7.39E-01\n",
      "  77 4.66E-06 3.37E-04 7.54E-01\n",
      "  78 4.55E-06 3.31E-04 7.68E-01\n",
      "  79 4.44E-06 3.25E-04 7.80E-01\n",
      "  80 4.33E-06 3.20E-04 7.93E-01\n",
      "  81 4.23E-06 3.14E-04 8.07E-01\n",
      "  82 4.14E-06 3.08E-04 8.19E-01\n",
      "  83 4.04E-06 3.03E-04 8.33E-01\n",
      "  84 3.95E-06 2.98E-04 8.50E-01\n",
      "  85 3.86E-06 2.93E-04 8.65E-01\n",
      "  86 3.78E-06 2.88E-04 8.80E-01\n",
      "  87 3.69E-06 2.83E-04 8.96E-01\n",
      "  88 3.61E-06 2.79E-04 9.10E-01\n",
      "  89 3.54E-06 2.74E-04 9.27E-01\n",
      "  90 3.46E-06 2.70E-04 9.41E-01\n",
      "  91 3.39E-06 2.66E-04 9.57E-01\n",
      "  92 3.32E-06 2.62E-04 9.70E-01\n",
      "  93 3.25E-06 2.58E-04 9.83E-01\n",
      "  94 3.19E-06 2.54E-04 9.96E-01\n",
      "  95 3.12E-06 2.50E-04 1.01E+00\n",
      "  96 3.06E-06 2.46E-04 1.02E+00\n",
      "  97 3.00E-06 2.42E-04 1.03E+00\n",
      "  98 2.94E-06 2.39E-04 1.04E+00\n",
      "  99 2.89E-06 2.35E-04 1.05E+00\n",
      " 100 2.83E-06 2.32E-04 1.06E+00\n",
      "===============================\n",
      "\n",
      "\n",
      "100\n",
      "[0.03449107 0.03449107]\n",
      "1.0574009418487549\n",
      "2.8304561425587446e-06\n",
      "0.0002321105148388349\n"
     ]
    }
   ],
   "source": [
    "# Set your optimality tolerance\n",
    "opt_tol = 1E-8\n",
    "# Set maximum optimizer iteration limit\n",
    "max_itr = 100\n",
    "\n",
    "prob = X4()\n",
    "\n",
    "# Set up your optimizer with your problem and pass in optimizer parameters\n",
    "# And declare outputs to be stored\n",
    "optimizer = SteepestDescent(prob,\n",
    "                            opt_tol=opt_tol,\n",
    "                            max_itr=max_itr,\n",
    "                            outputs=['itr', 'obj', 'x', 'opt', 'time'])\n",
    "\n",
    "# Check first derivatives at the initial guess, if needed\n",
    "optimizer.check_first_derivatives(prob.x.get_data())\n",
    "\n",
    "# Solve your optimization problem\n",
    "optimizer.solve()\n",
    "\n",
    "# Print results of optimization (summary_table contains information from each iteration)\n",
    "optimizer.print_results(summary_table=True)\n",
    "\n",
    "# Print any output that was declared\n",
    "# Since the arrays are long, here we only print the last entry and\n",
    "# verify it with the print_results() above\n",
    "\n",
    "print('\\n')\n",
    "print(optimizer.outputs['itr'][-1])\n",
    "print(optimizer.outputs['x'][-1])\n",
    "print(optimizer.outputs['time'][-1])\n",
    "print(optimizer.outputs['obj'][-1])\n",
    "print(optimizer.outputs['opt'][-1])"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "base",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.13"
  },
  "orig_nbformat": 4
 },
 "nbformat": 4,
 "nbformat_minor": 2
}